In [1]:
import pandas as pd
import numpy as np
import joblib
import re
from tqdm import tqdm

# Sklearn imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

## Load Data

In [2]:
file_path = 'data/final_df.csv' 
try:
    df_ori = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"*** ERROR: Cannot find {file_path} ***")


In [3]:
df = df_ori.copy()

# Sort by symbol and datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by=['symbol', 'datetime'])
df = df.reset_index(drop=True)

df['lret_1m'] = df.groupby('symbol')['close'].transform(lambda s: np.log(s).diff())
df['y_target'] = df.groupby('symbol')['lret_1m'].shift(-1)

initial_rows = len(df)
df = df[(df['y_target'].abs() <= 0.2)]
df = df.dropna(subset=['y_target', 'lret_1m'])

print(f"(Dropped {initial_rows - len(df)} unusal/NaN y value)")

# --- Re-sort by datatime, symbol ---
df = df.sort_values(by=['datetime', 'symbol'])
df = df.reset_index(drop=True)

df

(Dropped 12 unusal/NaN y value)


,datetime,symbol,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,f_minsin,f_mincos,...,split_nonpos_flag,shares_out,log_shares_out,eps_surp_pct_final,div_amount,log_shares_out_iqr_outlier,eps_estimate_rz_8,eps_actual,lret_1m,y_target
0,2024-04-30 12:51:00,AMAT,0,1,0,0,0,0,-0.220697,-0.975342,...,1,830897024,20.538016,0.00,0.0,0,0.0,0.00,-0.000199,-0.000100
1,2024-04-30 12:51:00,AMD,0,1,0,0,0,0,-0.220697,-0.975342,...,1,1616140032,21.203306,2.04,0.0,0,0.0,0.62,0.000314,-0.000126
2,2024-04-30 12:51:00,AVGO,0,1,0,0,0,0,-0.220697,-0.975342,...,1,465308000,19.958210,0.00,0.0,0,0.0,0.00,-0.000334,0.000721
3,2024-04-30 12:51:00,MU,0,1,0,0,0,0,-0.220697,-0.975342,...,1,1107369984,20.825254,0.00,0.0,0,0.0,0.00,0.000698,-0.000698
4,2024-04-30 12:51:00,NVDA,0,1,0,0,0,0,-0.220697,-0.975342,...,1,2500000000,21.639557,0.00,0.0,0,0.0,0.00,-0.000769,-0.000907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324883,2025-10-28 15:58:00,AMAT,0,1,0,0,0,0,-0.861629,-0.507538,...,1,796642427,20.495916,0.00,0.0,0,0.0,0.00,-0.000461,-0.000132
324884,2025-10-28 15:58:00,AMD,0,1,0,0,0,0,-0.861629,-0.507538,...,1,1633284837,21.213859,0.00,0.0,0,0.0,0.00,-0.000698,-0.001316
324885,2025-10-28 15:58:00,AVGO,0,1,0,0,0,0,-0.861629,-0.507538,...,1,4722365022,22.275576,0.00,0.0,0,0.0,0.00,0.000161,0.000375
324886,2025-10-28 15:58:00,MU,0,1,0,0,0,0,-0.861629,-0.507538,...,1,1122466035,20.838794,0.00,0.0,0,0.0,0.00,0.000270,-0.000135


In [4]:
df[["datetime", "symbol", 'fz_lret_1_rolling', "lret_1m", "y_target"]]

,datetime,symbol,fz_lret_1_rolling,lret_1m,y_target
0,2024-04-30 12:51:00,AMAT,-0.194539,-0.000199,-0.000100
1,2024-04-30 12:51:00,AMD,0.722399,0.000314,-0.000126
2,2024-04-30 12:51:00,AVGO,-0.248127,-0.000334,0.000721
3,2024-04-30 12:51:00,MU,0.793029,0.000698,-0.000698
4,2024-04-30 12:51:00,NVDA,-1.007944,-0.000769,-0.000907
...,...,...,...,...,...
324883,2025-10-28 15:58:00,AMAT,-0.714656,-0.000461,-0.000132
324884,2025-10-28 15:58:00,AMD,-0.811072,-0.000698,-0.001316
324885,2025-10-28 15:58:00,AVGO,-0.057482,0.000161,0.000375
324886,2025-10-28 15:58:00,MU,0.281664,0.000270,-0.000135


## Data Cleaning

In [5]:
y = df['y_target']
X = df.drop(columns=[
    'y_target', 'lret_1m', 'datetime', 'symbol',  
    'year', 'month', 'day', 'minute', 'minute_of_day'
], errors='ignore')

print(f"X original shape: {X.shape}")

# --- Find bad Cols with wrong data ---
desc = X.describe(percentiles=[0.99]).T
bad_cols = []

# std > 1000
bad_cols += desc.index[desc['std'] > 1e3].tolist()
# 99% > 1000
bad_cols += desc.index[desc['99%'].abs() > 1e3].tolist()
# Max > 1,000,000
bad_cols += desc.index[desc['max'].abs() > 1e6].tolist()
# Constant columns
bad_cols += desc.index[desc['std'] == 0].tolist()

bad_cols_set = sorted(set(bad_cols))
print(f"--- Dropping {len(bad_cols_set)} bad cols ---")
for col in bad_cols_set:
    print(f"  - {col}")

# --- Drop bad cols ---
X_cleaned = X.drop(columns=bad_cols_set)

print(f"\n--- X cleaned ---")
print(f"X (cleaned) shape: {X_cleaned.shape}")

X original shape: (324888, 184)
--- Dropping 37 bad cols ---
  - cnt_15m_rs
  - cnt_30m_rs
  - cnt_5m_rs
  - cnt_60m_rs
  - div_negative_flag
  - dow_5
  - dow_6
  - eps_actual_iqr_outlier
  - eps_estimate_iqr_outlier
  - fz_vol_ratio_60
  - mins_since_last_news_rs
  - morning_n_rs
  - morning_source_div_rs
  - morning_tone_mean_rs
  - morning_tone_sum_rs
  - n_news_ewm_hl15_is_zero
  - n_news_ewm_hl15_rs
  - n_news_ewm_hl5_is_zero
  - n_news_ewm_hl5_rs
  - n_news_rs
  - n_pos_raw_rs
  - overnight_n_rs
  - overnight_source_div_rs
  - overnight_tone_mean_rs
  - overnight_tone_sum_rs
  - shares_out
  - shares_out_ffill
  - shares_out_nonpos_flag
  - split_flag
  - surprise_tone_mean_rs
  - tone_ewm15_rs
  - tone_ewm30_rs
  - tone_mean_delta_rs
  - tone_mean_ewm_hl15_rs
  - tone_mean_rs
  - tone_sum_rs
  - volume

--- X cleaned ---
X (cleaned) shape: (324888, 147)


## Split Data

In [6]:
split_ratio = 1.0 / 1.5 
split_index = int(len(X_cleaned) * split_ratio)

X_val = X_cleaned.iloc[:split_index]
y_val = y.iloc[:split_index]

X_test = X_cleaned.iloc[split_index:]
y_test = y.iloc[split_index:]

print(f"--- Data Splitting done ---")
print(f" Validation set (X_val, y_val) shape: {X_val.shape}, {y_val.shape}")
print(f" Testing set (X_test, y_test) shape: {X_test.shape}, {y_test.shape}")

--- Data Splitting done ---
 Validation set (X_val, y_val) shape: (216592, 147), (216592,)
 Testing set (X_test, y_test) shape: (108296, 147), (108296,)


## Grid Search CV With MSE

In [7]:
class QuantileClipper(BaseEstimator, TransformerMixin):
    def __init__(self, lower=0.005, upper=0.995):
        self.lower = lower
        self.upper = upper
        self.q_low_ = None
        self.q_high_ = None

    def fit(self, X, y=None):
        self.q_low_  = np.nanpercentile(X, self.lower * 100, axis=0) 
        self.q_high_ = np.nanpercentile(X, self.upper * 100, axis=0)
        return self

    def transform(self, X):
        return np.clip(X, self.q_low_, self.q_high_)

In [8]:
## Pipeline
pipe = Pipeline([
    ("imp",  SimpleImputer(strategy="median")),
    ("clip", QuantileClipper(lower=0.005, upper=0.995)),
    ("sc",   MaxAbsScaler()),
    ("reg",  TransformedTargetRegressor(
                regressor=Ridge(solver="lsqr", max_iter=10000),
                transformer=StandardScaler())
    ),
])

# Hyper param grid
param_grid = {
    "reg__regressor__alpha": [10.0, 100.0, 1000.0, 1e4, 1e5]
}

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=10)

# Grid Search
print(f"\n--- GridSearchCV ... ---")
search = GridSearchCV(pipe, param_grid, 
                      cv=tscv,
                      scoring=["neg_mean_squared_error", 'r2'],
                      refit='neg_mean_squared_error',
                      n_jobs=-1, verbose=1)
search.fit(X_val, y_val)

# Result
best_alpha = search.best_params_["reg__regressor__alpha"]
best_score = -search.best_score_ 

print(f"Best Alpha (reg__regressor__alpha): {best_alpha}")
print(f"Best MSE: {best_score}")

results_df = pd.DataFrame(search.cv_results_)

columns_to_see = [
    'param_reg__regressor__alpha',
    'mean_test_neg_mean_squared_error',
    'std_test_neg_mean_squared_error',
    'mean_test_r2', 
    'std_test_r2'
]

columns_to_see = [col for col in columns_to_see if col in results_df.columns]

print("\n--- CV Details ---")
with pd.option_context('display.float_format', '{:,.8f}'.format):
    print(results_df[columns_to_see].sort_values(by='mean_test_neg_mean_squared_error', ascending=False))

# Save model
# joblib.dump(search.best_estimator_, 'final_ridge_pipeline.pkl')


--- GridSearchCV ... ---
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Alpha (reg__regressor__alpha): 100000.0
Best MSE: 6.991028792365208e-06

--- CV Details ---
  param_reg__regressor__alpha  mean_test_neg_mean_squared_error  \
4            100,000.00000000                       -0.00000699   
3             10,000.00000000                       -0.00000699   
2              1,000.00000000                       -0.00000700   
1                100.00000000                       -0.00000702   
0                 10.00000000                       -0.00000703   

   std_test_neg_mean_squared_error  mean_test_r2  std_test_r2  
4                       0.00000318   -0.00029473   0.00054315  
3                       0.00000319   -0.00082162   0.00117831  
2                       0.00000319   -0.00234332   0.00227063  
1                       0.00000319   -0.00445376   0.00447422  
0                       0.00000319   -0.00608973   0.00659787  


## Grid Search CV with Sharp Ratio

In [9]:
LASSO_SELECTED_FEATURES = [
    'f_minsin', 'f_mincos', 'close', 'f_volume_norm', 'low', 'oc_change',
    'rsi_14_clip', 'vol_60', 'morning_tone_sum', 'mins_since_last_news',
    'overnight_tone_mean', 'eps_estimate', 'shares_out_roll_std_252',
    '5d_before_earnings_flag', 'cnt_15m_is_zero', 'cnt_30m_is_zero',
    'cnt_60m_is_zero', 'dow_0', 'dow_1', 'dow_3', 'earnings_flag',
    'f_volu_z_15_clipped', 'f_volu_z_5_clipped', 'f_volu_z_60_clipped',
    'fz_dev_close_ema200', 'fz_dev_close_vwap_15', 'fz_dev_close_vwap_60',
    'fz_dev_tp_ema20', 'fz_hl_over_atr20', 'fz_lret_15_rolling',
    'fz_lret_1_rolling', 'fz_lret_30_rolling', 'fz_lret_5_rolling',
    'fz_lret_60_rolling', 'fz_macd_hist', 'fz_rsi_pos',
    'fz_spill_AMAT_lret_lag1', 'fz_spill_AMAT_lret_lag2',
    'fz_spill_AMAT_lret_lag3', 'fz_spill_AMAT_lret_lag4',
    'fz_spill_AMAT_lret_lag5', 'fz_spill_AMD_lret_lag1',
    'fz_spill_AMD_lret_lag2', 'fz_spill_AMD_lret_lag3',
    'fz_spill_AMD_lret_lag4', 'fz_spill_AMD_lret_lag5',
    'fz_spill_AVGO_lret_lag1', 'fz_spill_AVGO_lret_lag2',
    'fz_spill_AVGO_lret_lag4', 'fz_spill_AVGO_lret_lag5',
    'fz_spill_MU_lret_lag1', 'fz_spill_MU_lret_lag2',
    'fz_spill_MU_lret_lag3', 'fz_spill_MU_lret_lag4',
    'fz_spill_MU_lret_lag5', 'fz_spill_NVDA_lret_lag1',
    'fz_spill_NVDA_lret_lag2', 'fz_spill_NVDA_lret_lag3',
    'fz_spill_NVDA_lret_lag4', 'fz_spill_NVDA_lret_lag5', 'fz_vol_15',
    'fz_vol_30', 'fz_vol_5', 'fz_vol_60', 'fz_vol_ratio_15',
    'fz_vol_ratio_30', 'fz_vol_ratio_5', 'log_shares_out', 'minute_cos',
    'overnight_n_is_zero', 'shares_out_roll_z_252', 'split_mult',
    'split_ratio', 'tone_sum', 'volu_z_60'
]

existing_features = [col for col in LASSO_SELECTED_FEATURES if col in X_val.columns]


print(f"From {len(X_val.columns)} features reduced to {len(existing_features)} (Lasso Selected)")

X_val = X_val[existing_features]
X_test = X_test[existing_features]

print(f"X_val new shape: {X_val.shape}")
print(f"X_test new shape: {X_test.shape}")

# print(list(set(LASSO_SELECTED_FEATURES) - set(existing_features)))

From 147 features reduced to 75 (Lasso Selected)
X_val new shape: (216592, 75)
X_test new shape: (108296, 75)


### Applied Scaler only to raw features

In [10]:
def split_columns_for_scaling(X: pd.DataFrame):
    # Regex to find already-processed columns
    patterns = [
        r'^fz_', r'_z_', r'(?:^|_)tanh', r'(?:^|_)arctanh',
        r'(?:^|_)sin$', r'(?:^|_)cos$', r'^dow_\d+$',
        r'^has_news$', r'(?:^|_)log', r'(?:^|_)ln', r'(?:^|_)ewm',
        r'^tone_', r'_flag$', r'_iqr_outlier$'
    ]
    regex = re.compile('|'.join(patterns))

    name_matched = [c for c in X.columns if regex.search(c)]
    # Find all 0/1 binary columns
    binary_cols = [c for c in X.columns if X[c].nunique(dropna=True) <= 2]

    no_scale_cols = sorted(set(name_matched + binary_cols))
    scale_cols = [c for c in X.columns if c not in no_scale_cols]
    return no_scale_cols, scale_cols

# --- Final Pipeline ---
def build_final_pipe(alpha: float, preprocessor: ColumnTransformer) -> Pipeline:
    return Pipeline([
        ("preprocessor", preprocessor),
        ("reg",  TransformedTargetRegressor(
                    regressor=Ridge(solver="auto", alpha=alpha),
                    transformer=StandardScaler() # (Keep y-scaling)
                )
        ),
    ])

# --- Sharpe Ratio Optimization Loop ---
def calculate_sharpe(pnl_series: pd.Series) -> float:
    std = pnl_series.std()
    return 0.0 if std == 0 else pnl_series.mean() / std

In [11]:
no_scale_cols, scale_cols = split_columns_for_scaling(X_val)

print(f"--- Feature separation complete ---")
print(f"Will scale {len(scale_cols)} 'raw' columns (e.g., 'close', 'high'...)")
print(f"Will skip {len(no_scale_cols)} 'processed' columns (e.g., 'fz_...', 'dow_0'...)")

# --- Preprocessor (ColumnTransformer) ---

# Branch 1: Pipeline for "raw" columns
raw_branch_pipeline = Pipeline([
    ("clip", QuantileClipper(lower=0.005, upper=0.995)),
    ("sc",   MaxAbsScaler()),
])

# Preprocessor: Combines Branch 1 and the "skip" branch
preprocessor = ColumnTransformer(
    transformers=[
        # ("name", pipeline, columns_to_apply_to)
        ("raw_scaler", raw_branch_pipeline, scale_cols),
        ("passthrough_features", "passthrough", no_scale_cols),
    ],
    remainder="drop" # Safety net
)

alphas_to_test = [1.0, 10.0, 100.0, 1000.0, 1e4] 
tscv = TimeSeriesSplit(n_splits=10)

print("\n--- Running Manual CV (using ColumnTransformer) ---")
validation_results = {}

for alpha in alphas_to_test:
    print(f"  Testing Alpha: {alpha} ...")
    pipe = build_final_pipe(alpha, preprocessor)

    fold_pnls = []
    for train_idx, val_idx in tscv.split(X_val):
        X_tr, X_te = X_val.iloc[train_idx], X_val.iloc[val_idx]
        y_tr, y_te = y_val.iloc[train_idx], y_val.iloc[val_idx]

        pipe.fit(X_tr, y_tr)
        preds = pipe.predict(X_te)

        signals = np.sign(preds)
        pnl = pd.Series(signals, index=y_te.index) * y_te
        fold_pnls.append(pnl)

    validation_results[alpha] = pd.concat(fold_pnls)

# --- 6. Print Final Results ---
print("\n--- CV Results (Sharpe Ratio) ---")
sharpe_scores = {a: calculate_sharpe(pnl) for a, pnl in validation_results.items()}
sharpe_df = pd.Series(sharpe_scores, name="Sharpe_Ratio").sort_values(ascending=False)
print(sharpe_df)

best_alpha_by_sharpe = sharpe_df.idxmax()
print(f"\nBest Alpha (SR): {best_alpha_by_sharpe}")
print(f"Best SR: {sharpe_df.max():.6f}")

--- Feature separation complete ---
Will scale 13 'raw' columns (e.g., 'close', 'high'...)
Will skip 62 'processed' columns (e.g., 'fz_...', 'dow_0'...)

--- Running Manual CV (using ColumnTransformer) ---
  Testing Alpha: 1.0 ...
  Testing Alpha: 10.0 ...
  Testing Alpha: 100.0 ...
  Testing Alpha: 1000.0 ...
  Testing Alpha: 10000.0 ...

--- CV Results (Sharpe Ratio) ---
100.0      0.001263
10.0       0.000900
1.0        0.000890
10000.0    0.000618
1000.0    -0.000392
Name: Sharpe_Ratio, dtype: float64

Best Alpha (SR): 100.0
Best SR: 0.001263


### Fine Tune

In [12]:
no_scale_cols, scale_cols = split_columns_for_scaling(X_val)

print(f"--- Feature separation complete ---")
print(f"Will scale {len(scale_cols)} 'raw' columns (e.g., 'close', 'high'...)")
print(f"Will skip {len(no_scale_cols)} 'processed' columns (e.g., 'fz_...', 'dow_0'...)")

# --- Preprocessor (ColumnTransformer) ---

# Branch 1: Pipeline for "raw" columns
raw_branch_pipeline = Pipeline([
    ("clip", QuantileClipper(lower=0.005, upper=0.995)),
    ("sc",   MaxAbsScaler()),
])

# Preprocessor: Combines Branch 1 and the "skip" branch
preprocessor = ColumnTransformer(
    transformers=[
        # ("name", pipeline, columns_to_apply_to)
        ("raw_scaler", raw_branch_pipeline, scale_cols),
        ("passthrough_features", "passthrough", no_scale_cols),
    ],
    remainder="drop" # Safety net
)

# We will search the range [10, 500], focusing around 100.
alphas_to_test = [10.0, 25.0, 50.0, 75.0, 100.0, 150.0, 200.0, 300.0, 500.0]
tscv = TimeSeriesSplit(n_splits=10)

print("\n--- Running Manual CV (using ColumnTransformer) ---")
validation_results = {}

for alpha in alphas_to_test:
    print(f"  Testing Alpha: {alpha} ...")
    pipe = build_final_pipe(alpha, preprocessor)

    fold_pnls = []
    for train_idx, val_idx in tscv.split(X_val):
        X_tr, X_te = X_val.iloc[train_idx], X_val.iloc[val_idx]
        y_tr, y_te = y_val.iloc[train_idx], y_val.iloc[val_idx]

        pipe.fit(X_tr, y_tr)
        preds = pipe.predict(X_te)

        signals = np.sign(preds)
        pnl = pd.Series(signals, index=y_te.index) * y_te
        fold_pnls.append(pnl)

    validation_results[alpha] = pd.concat(fold_pnls)

# --- 6. Print Final Results ---
print("\n--- CV Results (Sharpe Ratio) ---")
sharpe_scores = {a: calculate_sharpe(pnl) for a, pnl in validation_results.items()}
sharpe_df = pd.Series(sharpe_scores, name="Sharpe_Ratio").sort_values(ascending=False)
print(sharpe_df)

best_alpha_by_sharpe = sharpe_df.idxmax()
print(f"\nBest Alpha (SR): {best_alpha_by_sharpe}")
print(f"Best SR: {sharpe_df.max():.6f}")

--- Feature separation complete ---
Will scale 13 'raw' columns (e.g., 'close', 'high'...)
Will skip 62 'processed' columns (e.g., 'fz_...', 'dow_0'...)

--- Running Manual CV (using ColumnTransformer) ---
  Testing Alpha: 10.0 ...
  Testing Alpha: 25.0 ...
  Testing Alpha: 50.0 ...
  Testing Alpha: 75.0 ...
  Testing Alpha: 100.0 ...
  Testing Alpha: 150.0 ...
  Testing Alpha: 200.0 ...
  Testing Alpha: 300.0 ...
  Testing Alpha: 500.0 ...

--- CV Results (Sharpe Ratio) ---
75.0     0.001514
100.0    0.001263
150.0    0.001255
200.0    0.001209
50.0     0.001112
25.0     0.001001
300.0    0.000957
10.0     0.000900
500.0    0.000133
Name: Sharpe_Ratio, dtype: float64

Best Alpha (SR): 75.0
Best SR: 0.001514


## 30-minute Rolling Window Backtesting

In [13]:
BEST_ALPHA = 75.0
ROLLING_WINDOW_MINUTES = 30

preprocessor = ColumnTransformer(
    transformers=[
        ("raw_scaler", raw_branch_pipeline, scale_cols),
        ("passthrough_features", "passthrough", no_scale_cols),
    ],
    remainder="drop"
)

# X_val (first year) to provide training data for the first day of X_test (last 6 months)
df_backtest = df.set_index('datetime')

X_full_history = df_backtest[X_val.columns]
y_full_history = df_backtest['y_target']
meta_full_history = df_backtest[['symbol']]

test_original_indices = X_test.index
test_datetimes = df.loc[test_original_indices]['datetime'].unique()

test_datetimes = np.sort(test_datetimes)

print(f"--- Loading {len(test_datetimes)} min rolling window backtest ---")
print(f"Testing period: From {test_datetimes.min()} to {test_datetimes.max()}")
print(f"Rolling Window: {ROLLING_WINDOW_MINUTES} min")
print(f"Best Alpha: {BEST_ALPHA}")

# --- Rolling Backtest ---
results_list = []

for current_dt in tqdm(test_datetimes):
    
    # Set training window
    train_start_dt = current_dt - pd.Timedelta(minutes=ROLLING_WINDOW_MINUTES)
    train_end_dt   = current_dt - pd.Timedelta(minutes=1)
    
    # Training data
    X_tr = X_full_history.loc[train_start_dt : train_end_dt]
    y_tr = y_full_history.loc[train_start_dt : train_end_dt]
    
    # check for enough data (15 min for 5 stocks)
    if len(X_tr) < 15 * 5:
        continue
        
    # Test data
    X_te = X_full_history.loc[current_dt]
    # make sure it is 2D
    if isinstance(X_te, pd.Series):
        X_te = X_te.to_frame().T 

    # Train a new model
    pipe = build_final_pipe(alpha=BEST_ALPHA, preprocessor=preprocessor)
    pipe.fit(X_tr, y_tr)
    
    # Prediction
    predictions = pipe.predict(X_te)
    
    # Save result into df
    meta_rows = meta_full_history.loc[current_dt]
    actuals = y_full_history.loc[current_dt]
    
    # if at t only has 1 stock
    if isinstance(meta_rows, pd.Series):
         results_list.append({
            "datetime": current_dt,
            "symbol": meta_rows['symbol'],
            "predicted_log_return": predictions[0],
            "actual_log_return": actuals
        })
    # if t has many stock
    else:
        for i in range(len(meta_rows)):
            results_list.append({
                "datetime": current_dt,
                "symbol": meta_rows.iloc[i]['symbol'],
                "predicted_log_return": predictions[i],
                "actual_log_return": actuals.iloc[i]
            })

# --- Final DataFrame ---
print("\n--- Backtesting done! Creating Final DataFrame... ---")
results_df = pd.DataFrame(results_list)

# --- Calculating Weight ---
# 2 types of weight

# sum(weights) = 1, Long-only
results_df['positive_prediction'] = results_df['predicted_log_return'].clip(lower=0)
minute_sum = results_df.groupby('datetime')['positive_prediction'].transform('sum')
results_df['weight_relative'] = np.where(
    minute_sum == 0, # if minute sum = 0
    0,               # .weight = 0 (hold)
    results_df['positive_prediction'] / minute_sum
)
results_df = results_df.drop(columns=['positive_prediction'])

# weight = sign(prediction)
results_df['weight_sign'] = np.sign(results_df['predicted_log_return'])

print("--- Final DataFrame created ---")
print(results_df.head())
print(f"\nTotal rows: {len(results_df)}")

--- Loading 21660 min rolling window backtest ---
Testing period: From 2025-04-29T15:59:00.000000000 to 2025-10-28T15:58:00.000000000
Rolling Window: 30 min
Best Alpha: 75.0


100%|██████████| 21660/21660 [03:42<00:00, 97.26it/s] 



--- Backtesting done! Creating Final DataFrame... ---
--- Final DataFrame created ---
             datetime symbol  predicted_log_return  actual_log_return  \
0 2025-04-29 15:59:00   AMAT             -0.000307          -0.014007   
1 2025-04-29 15:59:00    AMD             -0.000044          -0.006581   
2 2025-04-29 15:59:00   AVGO             -0.000359          -0.008008   
3 2025-04-29 15:59:00     MU              0.000013          -0.019444   
4 2025-04-29 15:59:00   NVDA             -0.000320          -0.025525   

   weight_relative  weight_sign  
0              0.0         -1.0  
1              0.0         -1.0  
2              0.0         -1.0  
3              1.0          1.0  
4              0.0         -1.0  

Total rows: 99749


In [14]:
results_df

,datetime,symbol,predicted_log_return,actual_log_return,weight_relative,weight_sign
0,2025-04-29 15:59:00,AMAT,-0.000307,-0.014007,0.000000,-1.0
1,2025-04-29 15:59:00,AMD,-0.000044,-0.006581,0.000000,-1.0
2,2025-04-29 15:59:00,AVGO,-0.000359,-0.008008,0.000000,-1.0
3,2025-04-29 15:59:00,MU,0.000013,-0.019444,1.000000,1.0
4,2025-04-29 15:59:00,NVDA,-0.000320,-0.025525,0.000000,-1.0
...,...,...,...,...,...,...
99744,2025-10-28 15:58:00,AMAT,0.000194,-0.000132,0.061321,1.0
99745,2025-10-28 15:58:00,AMD,0.000268,-0.001316,0.084617,1.0
99746,2025-10-28 15:58:00,AVGO,0.000898,0.000375,0.283711,1.0
99747,2025-10-28 15:58:00,MU,0.001046,-0.000135,0.330421,1.0


In [15]:
output_filename = 'data/ridge_result_df.csv'

results_df.to_csv(output_filename, index=False)

print(f"DataFrame saved to: {output_filename}")

DataFrame saved to: data/ridge_result_df.csv
